In [1]:
!nvidia-smi

Sun Apr 28 21:47:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
# %pip install cupy-cuda11x

In [5]:
import numpy as np
import cupy as cp

We have copy and pasted the functions defined in our pythontopython2.py below:

In [6]:
def setup(file_name):
    file = open(file_name + ".py", "a")
    file.write("import cupy")

def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
    return var

def numpy_add_to_py(a, b):
    a_name = get_var_name(a)
    b_name = get_var_name(b)
    return f"cupy.add({a_name}, {b_name})"

def numpy_subtract_to_py(a, b):
    a_name = get_var_name(a)
    b_name = get_var_name(b)
    return f"cupy.subtract({a_name}, {b_name})"

def numpy_sum_to_py(a):
    a_name = get_var_name(a)
    return f"cupy.sum({a_name})"

def numpy_dot_to_py(a, b):
    a_name = get_var_name(a)
    b_name = get_var_name(b)
    return f"cupy.dot({a_name}, {b_name})"

def array_to_py(variable_name, list_values):
    # list_values is a string containing comma-separated values
    # Turn list_values into a list, and each value in the list into an integer
    values = [int(value) for value in list_values.split(',')]

    return f"{variable_name} = cupy.array({values})"

We want to show firstly that these functions produce strings as expected.

In [7]:
setup('example_script')
# Expected to create or append to 'example_script.py' with 'import cupy'

In [8]:
x = np.array([10, 20, 30])
print(get_var_name(x))  # Should print 'x' if 'x' is in the global namespace

x


In [9]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(numpy_add_to_py(a, b))  # Expected output: "cupy.add(a, b)"

cupy.add(a, b)


In [10]:
a = np.array([7, 8, 9])
b = np.array([1, 2, 3])
print(numpy_subtract_to_py(a, b))  # Expected output: "cupy.subtract(a, b)"

cupy.subtract(a, b)


In [11]:
a = np.array([10, 20, 30])
print(numpy_sum_to_py(a))  # Expected output: "cupy.sum(a)"

cupy.sum(a)


In [12]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(numpy_dot_to_py(a, b))  # Expected output: "cupy.add(a, b)"

cupy.dot(a, b)


In [13]:
variable_name = 'data_array'
list_values = '100,200,300,400,500'
print(array_to_py(variable_name, list_values))  # Expected output: "data_array = cupy.array([100, 200, 300, 400, 500])"

data_array = cupy.array([100, 200, 300, 400, 500])


The next step would be to test the output strings actually run when they are  written to a file. In the next cell, we have copied all results from previous steps to emulate the file produced. We also want to compare the numpy vs. cupy run time.

In [14]:
import time

In [20]:
import numpy as np
import cupy as cp
import pandas as pd
import os

def time_operations(array_size, num_trials=3):
    # Initialize timing results
    times_gpu = []
    times_cpu = []

    for _ in range(num_trials):
        # Create arrays on GPU
        a_gpu = cp.random.rand(array_size)
        b_gpu = cp.random.rand(array_size)
        # Create arrays on CPU
        a_cpu = np.random.rand(array_size)
        b_cpu = np.random.rand(array_size)

        # Time CuPy operations
        start_gpu = time.time()
        cp.add(a_gpu, b_gpu)
        cp.subtract(a_gpu, b_gpu)
        cp.sum(a_gpu)
        times_gpu.append(time.time() - start_gpu)

        # Time NumPy operations
        start_cpu = time.time()
        np.add(a_cpu, b_cpu)
        np.subtract(a_cpu, b_cpu)
        np.sum(a_cpu)
        times_cpu.append(time.time() - start_cpu)

    # Average the times
    avg_time_gpu = sum(times_gpu) / num_trials
    avg_time_cpu = sum(times_cpu) / num_trials

    return avg_time_gpu, avg_time_cpu

array_sizes = [100000, 1000000, 10000000]  # Example sizes: 100K, 1M, 10M
results = []

for size in array_sizes:
    gpu_time, cpu_time = time_operations(size)
    results.append([size, gpu_time, cpu_time])

# Create DataFrame
df = pd.DataFrame(results, columns=['Array Size', 'CuPy Time (s)', 'NumPy Time (s)'])

# File path
csv_filename = 'performance_comparison.csv'

# Check if file exists to decide whether to write headers
file_exists = os.path.isfile(csv_filename)

# Append DataFrame to CSV (write header only if file does not exist)
df.to_csv(csv_filename, mode='a', header=not file_exists, index=False)

# Get the full path of the CSV file
full_path = os.path.abspath(csv_filename)
print(f"Results appended to {full_path}")

Results appended to /content/performance_comparison.csv
